# Clasificacion supervisada de Arboles de decision

In [1]:
# Importar paquete GEE
import ee
ee.Initialize()

## Filtro zona estudio mediante GEE

In [2]:
# Agregar un shp GEE
zona_ee = ee.FeatureCollection("users/bravomoralesnino/SHP/ZONA_19S")

In [3]:
# Importar geemap
import geemap
Map = geemap.Map(basemap='ESRI')

In [4]:
geometria = zona_ee.geometry()
Map.centerObject(geometria, 11)

In [5]:
Map.addLayer(zona_ee, {"color" : "00FF11"},name = "Zona estudio")
Map

Map(center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['position', 'transparent…

## Selección de imagen

In [6]:
# Seleccionar una imagen Landsat8
img_L8 = ee.Image("LANDSAT/LC08/C01/T1_SR/LC08_003069_20190808")\
             .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7'],
                     ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])\
             .multiply(0.0001)\
             .clip(zona_ee)\
             .reproject(crs="EPSG:32719", scale = 30)

In [7]:
# Nombre de las bandas
img_L8.bandNames().getInfo()

['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']

In [8]:
# Simbologia escala 0 - 1
viz_es = {
    'bands': ['SWIR1','NIR','RED'],
    'min': 0.1,
    'max': 0.6,
    'gamma': 1.6
}

In [9]:
Map.addLayer(img_L8, viz_es,name = "Landsat8 RS")
Map

Map(bottom=281402.6000366211, center=[-12.907261120236692, -70.0039174664375], controls=(WidgetControl(options…

## Agregar ROI estudio SHP

In [10]:
import os
os.getcwd() # Consultar ruta trabajo

'C:\\Users\\GEOMATICA\\Teledeteccion_Python\\GEE'

In [11]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\SHP\ROI"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\SHP\\ROI'

In [12]:
import geopandas as gpd

In [13]:
# cargar un shp
ROI = gpd.read_file("ROI_L8_Geo.shp")
ROI

,Id,Class,ID_Clas,geometry
0,1,Bosque,3,POINT (-69.95182 -12.96576)
1,2,Bosque,3,POINT (-69.95296 -12.94446)
2,3,Bosque,3,POINT (-69.95096 -12.93107)
3,4,Bosque,3,POINT (-69.90221 -12.93842)
4,5,Bosque,3,POINT (-69.88706 -12.93034)
...,...,...,...,...
195,197,Agricultura,1,POINT (-70.02166 -12.99661)
196,198,Agricultura,1,POINT (-70.01474 -12.99581)
197,199,Agricultura,1,POINT (-70.01503 -12.86148)
198,200,Agricultura,1,POINT (-70.00675 -12.85307)


In [14]:
# Crear un featureCollecion mediante GeoJSON
ee_ROI = ee.FeatureCollection(ROI.__geo_interface__)

In [15]:
Map.addLayer(ee_ROI,{"color" : "red"},name = "L8 ROI")
Map

Map(bottom=281403.0, center=[-12.907261120236692, -70.0039174664375], controls=(WidgetControl(options=['positi…

## Datos de entrenamiento ROI

In [16]:
# Obtener un FeatureCollection de datos de entrenamiento
training = img_L8.sampleRegions(collection = ee_ROI,
                                properties = ["ID_Clas","Class"],
                                scale = 30)

In [17]:
# seleccionar las bandas
bandas = ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']

## Parámetro clasificación Arboles desición

In [18]:
# image classification
classifier = ee.Classifier.smileCart().train(training, "ID_Clas", bandas)

## Aplicar clasificación

In [19]:
# Clasificador CART
clasificacion_CART = img_L8.classify(classifier)

In [20]:
# Simbologia clasificacion
viz_clas = {
    'palette': ["#4dde40","#2032ed",
                "#13800c","#dc0f0f"],
    'min': 1,
    'max': 4
}

In [21]:
Map.addLayer(clasificacion_CART, viz_clas,name = "Clasificado CART")
Map

Map(bottom=281405.0, center=[-12.908867400877059, -69.94720458984376], controls=(WidgetControl(options=['posit…

In [22]:
clasificacion_CART.bandNames().getInfo()

['classification']

## Matriz de confusión y precisión

In [23]:
# Estimar la matriz de confusión o error 
Matriz = classifier.confusionMatrix().getInfo()
Matriz

[[0, 0, 0, 0, 0],
 [0, 32, 0, 0, 0],
 [0, 0, 45, 0, 0],
 [0, 0, 0, 49, 0],
 [0, 0, 0, 0, 74]]

In [24]:
# precisión de la clasificación
classifier.confusionMatrix().accuracy().getInfo()

1

## Exportar Imagen 

In [25]:
os.getcwd()

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\SHP\\ROI'

In [26]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [27]:
# Exportar imagen Clasificado
geemap.ee_export_image(clasificacion_CART, filename="L8_clasificacion_CART.tif", scale=30, region=geometria, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE\L8_clasificacion_CART.tif
